## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Draw a Word Cloud Visualization based upon our Customer Review Sentiment


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- Create a word cloud based upon the themes detected in the customer reviews.
- Create a visualization for our postive and negative reviews:
    1. Execute a BigQuery SQL to gather our postive review data
    2. Execute a BigQuery SQL to gather our negative review data
    3. Create a postive word cloud image
    4. Create a negative word cloud image

## Word Cloud of Customer Review Themes

In [ ]:
import sys
!{sys.executable} -m pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
%%bigquery wordcloud_Positive_df

WITH max_data AS
(
  SELECT MAX(review_datetime) AS max_review_datetime
    FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review` AS customer_review
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.city` AS city
                ON city.city_id = 1
)
, theme_data AS
(
  SELECT CAST(JSON_VALUE(themes,'$.theme') AS STRING) AS review_theme
    FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review` AS customer_review
        CROSS JOIN UNNEST(JSON_QUERY_ARRAY(llm_detected_theme,'$')) AS themes
        CROSS JOIN max_data
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.city` AS city
                ON city.city_id = 1
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.location_history` AS location_history
                ON customer_review.location_id = location_history.location_id
                AND customer_review.review_datetime BETWEEN location_history.start_datetime AND location_history.stop_datetime
  WHERE customer_review.review_datetime BETWEEN TIMESTAMP_SUB(max_data.max_review_datetime, INTERVAL 7 DAY)
                                            AND max_data.max_review_datetime
    AND customer_review.review_sentiment = 'Positive'
)
SELECT STRING_AGG(REPLACE(review_theme,' ','-'), ' ') AS words
  FROM theme_data;

In [ ]:
%%bigquery wordcloud_Negative_df

WITH max_data AS
(
  SELECT MAX(review_datetime) AS max_review_datetime
    FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review` AS customer_review
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.city` AS city
                ON city.city_id = 1
)
, theme_data AS
(
  SELECT CAST(JSON_VALUE(themes,'$.theme') AS STRING) AS review_theme
    FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review` AS customer_review
        CROSS JOIN UNNEST(JSON_QUERY_ARRAY(llm_detected_theme,'$')) AS themes
        CROSS JOIN max_data
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.city` AS city
                ON city.city_id = 1
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.location_history` AS location_history
                ON customer_review.location_id = location_history.location_id
                AND customer_review.review_datetime BETWEEN location_history.start_datetime AND location_history.stop_datetime
  WHERE customer_review.review_datetime BETWEEN TIMESTAMP_SUB(max_data.max_review_datetime, INTERVAL 7 DAY)
                                            AND max_data.max_review_datetime
    AND customer_review.review_sentiment = 'Negative'
)
SELECT STRING_AGG(REPLACE(review_theme,' ','-'), ' ') AS words
  FROM theme_data;

### Postive Reviews


In [ ]:
wc = WordCloud(background_color='white', width = 1920, height = 1080).generate(wordcloud_Positive_df.iloc[0]['words'])
plt.rcParams['figure.figsize'] = [12, 12]
plt.axis("off")
plt.imshow(wc)

### Negative Reviews


In [ ]:
wc = WordCloud(background_color='white', width = 1920, height = 1080).generate(wordcloud_Negative_df.iloc[0]['words'])
plt.rcParams['figure.figsize'] = [12, 12]
plt.axis("off")
plt.imshow(wc)